Материалы:

- https://pyyaml.org/wiki/PyYAMLDocumentation
- https://realpython.com/python-yaml/#dumping-python-objects-to-yaml-documents
- https://matthewpburruss.com/post/yaml/

In [1]:
import sys
import yaml
from dataclasses import dataclass
sys.version

'3.10.4 (main, Jun 27 2022, 16:57:57) [GCC 11.2.0]'

In [2]:
@dataclass
class Person:
   first_name: str
   last_name: str

Subclassing `YAMLObject` is an easy way to define tags, constructors, and representers for your classes.

You only need to override the `yaml_tag` attribute. If you want to define your custom constructor and representer, redefine the `from_yaml` and `to_yaml` method correspondingly.

In [3]:
# yaml.YAMLObject uses metaclass magic to register a constructor,
# which transforms a YAML node to a class instance, and a representer,
# which serializes a class instance to a YAML node.

class PersonX(yaml.YAMLObject):
    yaml_tag = '!PersonX'
    yaml_loader = yaml.SafeLoader

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    def __repr__(self):
        return f'{type(self).__name__}({self.first_name}, {self.last_name})'

In [4]:
(person := Person('V', 'K'))

Person(first_name='V', last_name='K')

In [5]:
(person_x := PersonX('VX', 'KX'))

PersonX(VX, KX)

# Сериализация (dump)

In [6]:
yaml.dump(person), yaml.dump(person_x)

('!!python/object:__main__.Person\nfirst_name: V\nlast_name: K\n',
 '!PersonX\nfirst_name: VX\nlast_name: KX\n')

In [7]:
try:
    yaml.safe_dump(person)
except yaml.YAMLError as e:
    print(e)

('cannot represent an object', Person(first_name='V', last_name='K'))


# Десериализация (load)

In [8]:
s = """
    !!python/object:__main__.Person
    first_name: V
    last_name: K
"""

s_x = """
    !PersonX
    first_name: V
    last_name: K
"""

In [9]:
yaml.unsafe_load(s), yaml.load(s, Loader=yaml.UnsafeLoader)

(Person(first_name='V', last_name='K'), Person(first_name='V', last_name='K'))

In [10]:
try:
    yaml.unsafe_load(s_x)
except yaml.YAMLError as e:
    print(e)

could not determine a constructor for the tag '!PersonX'
  in "<unicode string>", line 2, column 5:
        !PersonX
        ^


In [11]:
try:
    yaml.safe_load(s)
except yaml.YAMLError as e:
    print(e)

could not determine a constructor for the tag 'tag:yaml.org,2002:python/object:__main__.Person'
  in "<unicode string>", line 2, column 5:
        !!python/object:__main__.Person
        ^


A python object can be marked as safe and thus be recognized by `yaml.safe_load`.

To do this, derive it from `yaml.YAMLObject` (as explained in section Constructors, representers, resolvers) and
explicitly set its class property `yaml_loader` to `yaml.SafeLoader`.

In [12]:
yaml.safe_load(s_x)

PersonX(V, K)

# Constructors, representers, resolvers

`yaml.YAMLObject` uses metaclass magic to register a constructor, which transforms a YAML node to a class instance, and a representer, which serializes a class instance to a YAML node.

If you don’t want to use metaclasses, you may register your constructors and representers using the functions `yaml.add_constructor` and `yaml.add_representer`.

You might not want to specify the tag !dice everywhere. There is a way to teach PyYAML that any untagged plain scalar which looks like XdY has the implicit tag !dice. Use `add_implicit_resolver`.

In [13]:
@dataclass
class Employee:
  """Employee class."""
  id: int
  name: str


def employee_constructor(loader: yaml.SafeLoader, node: yaml.nodes.MappingNode) -> Employee:
  """Construct an employee."""
  return Employee(**loader.construct_mapping(node))

def employee_representer(dumper: yaml.SafeDumper, emp: Employee) -> yaml.nodes.MappingNode:
  """Represent an employee instance as a YAML mapping node."""
  return dumper.represent_mapping('!Employee', {
    'name': emp.name,
    'id': emp.id,
  })


def get_loader():
  """Add constructors to PyYAML loader."""
  loader = yaml.SafeLoader
  loader.add_constructor('!Employee', employee_constructor)
  return loader

def get_dumper():
  """Add representers to a YAML seriailizer."""
  safe_dumper = yaml.SafeDumper
  safe_dumper.add_representer(Employee, employee_representer)
  return safe_dumper

d = """
    name: MyBusiness
    locations:
    - Hawaii
    - India
    - Japan
    employees:
    - !Employee
        name: Matthew Burruss
        id: 1
    - !Employee
        name: John Doe
        id: 2
"""
yaml.load(d, Loader=get_loader())


{'name': 'MyBusiness',
 'locations': ['Hawaii', 'India', 'Japan'],
 'employees': [Employee(id=1, name='Matthew Burruss'),
  Employee(id=2, name='John Doe')]}

In [14]:
yaml.SafeLoader

yaml.loader.SafeLoader

In [15]:
yaml.dump([Employee(1, 'Ivan'), Employee(2, 'Mike')], Dumper=get_dumper())

'- !Employee\n  id: 1\n  name: Ivan\n- !Employee\n  id: 2\n  name: Mike\n'